---
## 1. 화면 비율을 설정하고 필요한 라이브러리들을 불러온다.

In [1]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

##1. selenium, time, pandas 모듈을 불러온다.
from selenium import webdriver                                               # selenium webdriver 개체 필수 요소.
from selenium.webdriver.common.by import By                                  # 태그를 가져올 때 사용함.
from selenium.webdriver.support.ui import WebDriverWait                      # 특정 태그가 로드 될 때까지 기다리게 할 수 있음.
from selenium.webdriver.support import expected_conditions as EC             # 특정 태그가 로드 될 때까지 기다리게 할 수 있음.
from selenium.webdriver.common.keys import Keys                         
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup                                                # 셀레니움 가져온 html 파일을 BeautifulSoup로 파싱하고자 할 때.

import urllib.request                                                        # 이미지를 저장하기 위해 필요한 모듈.
import pickle                                                                # 객체를 저장.

import time
import pandas as pd
from pandas import DataFrame

# 이미지를 불러오지 않아 로딩이 빠르게 되게 하는 코드.
chrome_options = Options()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")

In [2]:
'''
querys = []
while True:
    inp = input("검색할 이미지를 입력해 주세요(그만 두고 싶을 때 공백 입력): ")
    if inp == "":
        break
    querys.append(inp)
print(querys)
'''
querys = ['투싼', '코나', '싼타페', '그랜저', '아반떼', '벨로스터']

---
## 2. 크롤링을 한다.

In [3]:
#크롬 웹브라우저를 통해 구글 이미지 검색 페이지를 열고 query에 따라 검색하고 저장한다.
for query in querys:
    driver = webdriver.Chrome("./chromedriver")
    driver.implicitly_wait(5)
    driver.get("https://www.google.co.kr/imghp?hl=ko&tab=wi")
    driver.find_element_by_css_selector("#lst-ib").send_keys(query)    # querys에 담긴 값을 하나씩 꺼내서 검색창에 입력한다.
    driver.find_element_by_name("btnG").click()                        # 검색 버튼을 클릭한다.
    
    print("'%s'를 검색 하였습니다." % query)

'''
이미지 주소를 파싱 해오는데, 만약 제대로 파싱이 안 되었으면 urls에 None이 들어간다.
따라서 예외처리 함수인 try~except를 통해 urls에서 'None'을 찾아본다.
만약 제대로 파싱 되었으면(== debug를 할 때 에러가 발생) except으로 들어가서 while문을 종료한다. (에러가 발생하면 제대로 된 것임... 헷갈리지 말자!)
만약 제대로 파싱이 안 되었으면 debug에 어떤 숫자 값이 저장된다. 이때는 다시 while문 루프를 돌아준다. (제대로 파싱이 될 때까지 파싱 반복.)
'''

    while True:
        try:
            imgs = driver.execute_script("return document.querySelectorAll('.rg_bx.rg_di.rg_el.ivg-i > a > img');")
            urls = [str(img.get_attribute('src')) for img in imgs]
            debug = urls.index('None')
            print("%s번 째 url이 제대로 파싱되지 않았습니다." % debug)
        except:
            print("이미지 url을 제대로 파싱 하였습니다.")
            break

    for url in urls:
        urllib.request.urlretrieve(url, filename="/mnt/new/hyundai/" + query + "/" + str(urls.index(url)) + ".jpg")

        if urls.index(url) % 20 == 0:
            print("%s:  %s(/%s)번 째까지 완료하였습니다." % (query, urls.index(url), len(urls)))

'투싼'를 검색 하였습니다.
'코나'를 검색 하였습니다.
'싼타페'를 검색 하였습니다.
'그랜저'를 검색 하였습니다.
'아반떼'를 검색 하였습니다.
'벨로스터'를 검색 하였습니다.


In [4]:
try:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='react-root']/section/main/article/div[2]/div[2]/p/a")))
except:
    print("페이지를 정상적으로 로드하지 못했습니다.")
finally:
    driver.find_element_by_xpath("//*[@id='react-root']/section/main/article/div[2]/div[2]/p/a").click()
    
time.sleep(1)

# 로그인 창이 뜰 때까지 기다린 후 아이디와 패스워드를 입력하여 로그인을 시도한다.
try:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "username")))
except:
    print("페이지를 정상적으로 로드하지 못했습니다.")
finally:
    driver.find_element_by_xpath("//*[@name='username']").send_keys(user_id)
    driver.find_element_by_xpath("//*[@name='password']").send_keys(user_pwd)
    driver.find_element_by_xpath("//*[@id='react-root']/section/main/div/article/div/div[1]/div/form/span/button").click()

# 로그인에 성공 했을 때 뜨는 알림창을 끈다.
driver.find_element_by_xpath("/html/body/div[3]/div/div/div/div[3]/button[2]").click()    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div/div/div[3]/button[2]"}
  (Session info: chrome=66.0.3359.181)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.12.6 x86_64)


---
## 4. 사용자의 입력을 받아서 검색한다.

In [10]:
result = pd.DataFrame([])
print(result)

Empty DataFrame
Columns: []
Index: []


In [6]:
query = str(input("검색어를 입력 해 주세요: "))
search = driver.find_element_by_xpath("//*[@id='react-root']/section/nav/div[2]/div/div/div[2]/input")
search.send_keys(query)

try:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='react-root']/section/nav/div[2]/div/div/div[2]/div[2]/div[2]")))
except:
    print("검색어가 제대로 입력되지 않았습니다.")
finally:
    search.send_keys(Keys.ENTER)
    search.send_keys(Keys.ENTER) # Keys.ENTER를 두번 해야 검색이 이루어지네... 원인은 모르겠음.

검색어를 입력 해 주세요: 제과


크롤링 한 결과를 담을 빈 데이터 프레임을 만든다.

---
## 7. URL을 이용하여 각 피드에 접속 후 writer, contents를 크롤링한다.

In [15]:
%%time

result_dt = pd.DataFrame([])
i = 1
for url in urls:
    try:
        driver.get(url)
        time.sleep(0.2)

        writer = driver.find_element_by_css_selector(".FPmhX.notranslate.TlrDj").text
        contents = driver.find_element_by_css_selector(".gElp9:nth-of-type(1) span").text
        tags = driver.find_elements_by_css_selector(".gElp9:nth-of-type(1) span a")
        tags_li = []
        for tag in tags:
            tags_li.append(tag.text)

        result_dt = result_dt.append(pd.DataFrame([[writer, contents, tags_li]], columns=["Writer", "Contents", "Tags"]), ignore_index=True)

        if i % 10 == 0:
            print("loop:", i, "/", len(urls), end=", ")
    except:
        print(i,"번 째 URL에서 에러가 발생하였습니다.")
    i += 1

driver.close()



loop: 10 / 15180, loop: 20 / 15180, loop: 30 / 15180, loop: 40 / 15180, loop: 50 / 15180, loop: 60 / 15180, loop: 70 / 15180, loop: 80 / 15180, loop: 90 / 15180, loop: 100 / 15180, loop: 110 / 15180, loop: 120 / 15180, loop: 130 / 15180, loop: 140 / 15180, loop: 150 / 15180, loop: 160 / 15180, loop: 170 / 15180, loop: 180 / 15180, loop: 190 / 15180, loop: 200 / 15180, loop: 210 / 15180, loop: 220 / 15180, loop: 230 / 15180, loop: 240 / 15180, loop: 250 / 15180, loop: 260 / 15180, loop: 270 / 15180, loop: 280 / 15180, loop: 290 / 15180, loop: 300 / 15180, loop: 310 / 15180, loop: 320 / 15180, loop: 330 / 15180, loop: 340 / 15180, loop: 350 / 15180, loop: 360 / 15180, loop: 370 / 15180, loop: 380 / 15180, loop: 390 / 15180, loop: 400 / 15180, loop: 410 / 15180, loop: 420 / 15180, loop: 430 / 15180, loop: 440 / 15180, loop: 450 / 15180, loop: 460 / 15180, loop: 470 / 15180, loop: 480 / 15180, loop: 490 / 15180, loop: 500 / 15180, loop: 510 / 15180, loop: 520 / 15180, loop: 530 / 15180, lo

loop: 7790 / 15180, loop: 7800 / 15180, loop: 7810 / 15180, loop: 7820 / 15180, loop: 7830 / 15180, loop: 7840 / 15180, loop: 7850 / 15180, loop: 7860 / 15180, loop: 7870 / 15180, loop: 7880 / 15180, loop: 7890 / 15180, loop: 7900 / 15180, loop: 7910 / 15180, loop: 7920 / 15180, loop: 7930 / 15180, loop: 7940 / 15180, loop: 7950 / 15180, loop: 7960 / 15180, loop: 7970 / 15180, loop: 7980 / 15180, loop: 7990 / 15180, 7995 번 째 URL에서 에러가 발생하였습니다.
loop: 8000 / 15180, loop: 8010 / 15180, loop: 8020 / 15180, loop: 8030 / 15180, loop: 8040 / 15180, loop: 8050 / 15180, loop: 8060 / 15180, loop: 8070 / 15180, loop: 8080 / 15180, loop: 8090 / 15180, loop: 8100 / 15180, loop: 8110 / 15180, loop: 8120 / 15180, loop: 8130 / 15180, loop: 8140 / 15180, loop: 8150 / 15180, 8154 번 째 URL에서 에러가 발생하였습니다.
loop: 8160 / 15180, 8167 번 째 URL에서 에러가 발생하였습니다.
loop: 8170 / 15180, loop: 8180 / 15180, loop: 8190 / 15180, loop: 8200 / 15180, loop: 8210 / 15180, loop: 8220 / 15180, loop: 8230 / 15180, loop: 8240 / 151

loop: 14350 / 15180, loop: 14360 / 15180, loop: 14370 / 15180, loop: 14380 / 15180, loop: 14390 / 15180, loop: 14400 / 15180, loop: 14410 / 15180, loop: 14420 / 15180, loop: 14430 / 15180, loop: 14440 / 15180, loop: 14450 / 15180, loop: 14460 / 15180, loop: 14470 / 15180, loop: 14480 / 15180, loop: 14490 / 15180, loop: 14500 / 15180, loop: 14510 / 15180, loop: 14520 / 15180, loop: 14530 / 15180, loop: 14540 / 15180, loop: 14550 / 15180, loop: 14560 / 15180, loop: 14570 / 15180, loop: 14580 / 15180, loop: 14590 / 15180, loop: 14600 / 15180, loop: 14610 / 15180, loop: 14620 / 15180, loop: 14630 / 15180, loop: 14640 / 15180, loop: 14650 / 15180, loop: 14660 / 15180, loop: 14670 / 15180, loop: 14680 / 15180, loop: 14690 / 15180, loop: 14700 / 15180, loop: 14710 / 15180, loop: 14720 / 15180, loop: 14730 / 15180, loop: 14740 / 15180, loop: 14750 / 15180, loop: 14760 / 15180, loop: 14770 / 15180, loop: 14780 / 15180, loop: 14790 / 15180, loop: 14800 / 15180, loop: 14810 / 15180, loop: 14820 /

In [16]:
with open("result_dt_" + query + ".pickle", "wb") as f:
    pickle.dump(result_dt, f)

In [17]:
result_dt.to_clipboard()

In [18]:
'''
with open("rec_list.pickle", "wb") as f:
    pickle.dump(rec_list, f)
'''

'\nwith open("rec_list.pickle", "wb") as f:\n    pickle.dump(rec_list, f)\n'

In [19]:
'''
with open("rec_list.piccle", "rb") as f:
    data = pickle.load(f)
    print(data)
'''

'\nwith open("rec_list.piccle", "rb") as f:\n    data = pickle.load(f)\n    print(data)\n'